# C2Q Method 1

In [1]:
import numpy as np
from IPython.display import Markdown as md

# Random data function used in experiments
def random_data(n_qubits, magnitude=False, use_complex_data=False, seed=None):
    if seed:
        np.random.seed(seed)

    n_states = 2**n_qubits
    x = np.random.rand(n_states)

    if use_complex_data:
        x = x + (np.random.rand(n_states) * 2j) - 1j
    else:
        x = x * 255

    mag = np.linalg.norm(x)
    x_in = x / mag

    if magnitude:
        return x_in, mag
    else:
        return x_in
    
def partial_measurement(psi_in, bits_to_remove):
    psi_out = np.zeros_like(psi_in)  # Output vector with same length as psi_in

    # Create bitmask from bits_to_remove
    bitmask = sum(2**x for x in bits_to_remove)
    bitmask = ~bitmask  # Negative mask of bitmask

    # Reduce psi_in while preserving indices
    for i, x in enumerate(psi_in):
        i_out = i & bitmask  # index in psi_out to insert value
        psi_out[i_out] += (
            np.abs(x) ** 2
        )  # Add the square of the value in psi (probability)
    psi_out = np.sqrt(psi_out)  # Take sqrt to return to a "statevector"

    return psi_out

# x_in = random_data(n_qubits=2, use_complex_data=False, seed=2)
# x_in = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0]

theta1 = np.pi/4
theta2 = 3*np.pi/4
x_in = [np.cos(theta1/2), np.sin(theta1/2), np.cos(theta2/2), np.sin(theta2/2)]
x_in = partial_measurement(x_in, [1])
x_in = x_in[:2]

print(x_in, theta1, theta2)

[1.16342313 0.80401904] 0.7853981633974483 1.5707963267948966


In [2]:
# Format input array properly
x_in = np.array(x_in)

# Calculate number of necessary qubits and states
num_qubits = int(np.ceil(np.log2(len(x_in))))
num_states = np.power(2, num_qubits, dtype=int)

# Pad input array if necessary
if num_states - len(x_in) != 0: 
    print(f'Input length is not power of 2. Padding...')
    x_in = np.pad(x_in, (0, num_states-len(x_in)), 'constant', constant_values=0)
    print(f'New input: {x_in}')

# Normalize input array if necessary
# if np.linalg.norm(x_in) != 1:
#     print(f'Input is not normalized. Normalizing...')
#     x_in = x_in / np.linalg.norm(x_in)
#     print(f'New input: {x_in}')

In [3]:
### ============================ ###
###  Calculate input parameters  ###
### ============================ ###

# Default values
n = int(num_states/2)
theta_array = np.empty(n)
phi_array = np.empty(n)
r_array = np.empty(n)
t_array = np.empty(n)

x_new = np.reshape(x_in, (int(len(x_in)/2), 2))

for i, (alpha, beta) in enumerate(x_new):
    alpha_mag = np.abs(alpha)
    alpha_phase = np.angle(alpha)
    beta_mag = np.abs(beta)
    beta_phase = np.angle(beta)

    with np.errstate(divide='ignore'):
        theta_array[i] = 2*np.arctan(beta_mag/alpha_mag)
    phi_array[i] = beta_phase - alpha_phase
    #r_array[i] = np.sqrt(alpha_mag**2 + beta_mag**2)
    r_array[i] = np.sqrt(alpha_mag**2 + beta_mag**2) * np.power(np.sqrt(2), num_qubits-1)
    t_array[i] = beta_phase + alpha_phase

In [4]:
# Basic gates

def Ry(theta):
    return np.matrix(f'{np.cos(theta/2)}, {-np.sin(theta/2)}; {np.sin(theta/2)}, {np.cos(theta/2)}')

def Rz(phi):
    return np.matrix(f'{np.exp(-1j*phi/2)}, {0}; {0}, {np.exp(1j*phi/2)}')

def Uj(theta, phi, r, t):
    #return r * np.exp(1j*t/2) * (Rz(phi) @ Ry(theta))
    return (Rz(phi) @ Ry(theta) @ Rz(-t)) * r    

In [5]:
# Construct each Uj (gates that are multiple contolled)
Uj_array = []

for _, (theta, phi, r, t) in enumerate(zip(theta_array, phi_array, r_array, t_array)):
    U_j = Uj(theta, phi, r, t)
    Uj_array.append(U_j)

In [6]:
# Construct entire Ublock matrix (entire circuit past H gates)
U_block = np.zeros((num_states, num_states), dtype=complex)

for i, U_j in enumerate(Uj_array):
    U_block[2*i:2*i+2, 2*i:2*i+2] = U_j

In [7]:
# 1 qubit |0> state
zero_state = np.matrix('1;0')

# Create initial quantum state
psi_0 = zero_state
for _ in range(num_qubits-1):
    psi_0 = np.kron(zero_state, psi_0)

# basic H gate
Hadamard = np.matrix('1 1; 1 -1') / np.sqrt(2)

# Add H gates to initial state
H_kron = np.eye(2)
for i in reversed(range(num_qubits-1)):
    H_kron = np.kron(Hadamard, H_kron)

psi_0 = H_kron @ psi_0

In [8]:
# Perform encoding on circuit

# The sqrt(2) term is needed to counter the one added by Hadamard gates
# Should this be considered part of r?
#psi = np.power(np.sqrt(2), num_qubits-1) * (U_block @ psi_0)
psi = (U_block @ psi_0)

#U_overall = np.power(np.sqrt(2), num_qubits-1) * (U_block @ H_kron)
U_overall = (U_block @ H_kron)

In [9]:
# Helper function for printing matrix in LATEX format
def matrix(M: np.matrix) -> str:
    output = '\\begin{bmatrix}'

    n_cols = 1
    if len(M.shape) == 2:
        _, n_cols = M.shape

    for i, m in enumerate(np.nditer(M)):
        output += f'{m:.03f}'

        if ((i+1) % n_cols) == 0:
            output += '\\\\'
        else:
            output += '&'

    output += '\\end{bmatrix}'

    return output

# Test if matrices are unitary
def is_unitary(M: np.matrix) -> bool:
    M = np.asmatrix(M)

    if len(M.shape) != 2:
        return False;

    n_rows, n_cols = M.shape

    if n_rows != n_cols:
        return False

    M_squared = M @ M.getH()
    M_squared = M_squared.round(3)

    # print(M_squared)

    return np.array_equal(M_squared, np.eye(n_rows))

In [10]:
# Print important values in LATEX

md_str = f'Input data: ${matrix(x_in)}$, '
md_str += f'Number of qubits: {num_qubits}\n'

md_str += '\n'

md_str += 'List of parameters: \n\n'
md_str += f'$\\theta = \pi{matrix(theta_array/np.pi)}$, '
md_str += f'$\phi = {matrix(phi_array)}$, '
md_str += f'$r = {matrix(r_array)}$, '
md_str += f'$t = {matrix(t_array)}$\n'

md_str += '\n'

md_str += 'Each $U_j$ matrix defined in C2Q Method 1: \n\n'
for j, Uj in enumerate(Uj_array):
    md_str += f'$U_{j} = {matrix(Uj)}$\n\n'

md_str += '\n'

md_str += 'The state after the Hadamard gates: '
md_str += f'$\left( H^{{\otimes (n-1)}} \otimes I \\right) \lvert 0 \\rangle^{{\otimes n}} = \\vert \Psi_0 \\rangle$ \n\n'
md_str += f'$\lvert \Psi_0 \\rangle = {matrix(psi_0)}$\n'

md_str += '\n'

md_str += 'The overall operator after the Hadamard gates: '
md_str += f'$\left( U_{{block}}\lvert \Psi_0 \\rangle = \\vert \Psi \\rangle \\right)$ \n\n'
md_str += f'$U_{{block}} = {matrix(U_block)}$\n'
md_str += f'(Unitary: {is_unitary(U_block)})\n\n'

md_str += '\n'

md_str += f'The matrix for the circuit overall: '
md_str += f'$\left( U^{{C2Q-1}}\lvert 0 \\rangle^{{\otimes n}} = \\vert \Psi \\rangle \\right)$ \n\n'
md_str += f'$U^{{C2Q-1}} = {matrix(U_overall)}$\n'
md_str += f'(Unitary: {is_unitary(U_overall)})\n\n'

md_str += '\n'

md_str += 'Final encoded state: \n\n'
md_str += f'$\lvert \Psi \\rangle = {matrix(psi)}$\n'

md(md_str)

Input data: $\begin{bmatrix}1.163\\0.804\\\end{bmatrix}$, Number of qubits: 1

List of parameters: 

$\theta = \pi\begin{bmatrix}0.385\\\end{bmatrix}$, $\phi = \begin{bmatrix}0.000\\\end{bmatrix}$, $r = \begin{bmatrix}1.414\\\end{bmatrix}$, $t = \begin{bmatrix}0.000\\\end{bmatrix}$

Each $U_j$ matrix defined in C2Q Method 1: 

$U_0 = \begin{bmatrix}1.163+0.000j&-0.804+0.000j\\0.804+0.000j&1.163+0.000j\\\end{bmatrix}$


The state after the Hadamard gates: $\left( H^{\otimes (n-1)} \otimes I \right) \lvert 0 \rangle^{\otimes n} = \vert \Psi_0 \rangle$ 

$\lvert \Psi_0 \rangle = \begin{bmatrix}1.000\\0.000\\\end{bmatrix}$

The overall operator after the Hadamard gates: $\left( U_{block}\lvert \Psi_0 \rangle = \vert \Psi \rangle \right)$ 

$U_{block} = \begin{bmatrix}1.163+0.000j&-0.804+0.000j\\0.804+0.000j&1.163+0.000j\\\end{bmatrix}$
(Unitary: False)


The matrix for the circuit overall: $\left( U^{C2Q-1}\lvert 0 \rangle^{\otimes n} = \vert \Psi \rangle \right)$ 

$U^{C2Q-1} = \begin{bmatrix}1.163+0.000j&-0.804+0.000j\\0.804+0.000j&1.163+0.000j\\\end{bmatrix}$
(Unitary: False)


Final encoded state: 

$\lvert \Psi \rangle = \begin{bmatrix}1.163+0.000j\\0.804+0.000j\\\end{bmatrix}$
